$\newcommand{\myvector}[2]{\left\{ \begin{array}{c}#1\\#2\end{array} \right\}}$
$\newcommand{\mymatrix}[4]{\left[ \begin{array}{c}#1&#2\\#3&#4\end{array} \right]}$
$\newcommand{\sM}[1]{\boldsymbol{s}_{\mathrm{M#1}}}$
$\newcommand{\rvector}[1]{\boldsymbol{r}_{\mathrm{#1}}}$

# APPENDIX F:  Effect of informative priors


This Appendix revists Appendix D to consider the effect of informative priors.

Let's first assemble all relevant functions from Appendix D. Of these functions, the only function that has been changed relative to Appendix D is `solution_bayesian`, which here offer three sets of prior distribution options: (1) vague, (2) informative and (3) highly informative.
<br>
<br>



In [1]:

from math import sin,cos,radians,degrees
import numpy as np
from scipy import optimize
import pymc


def rotation_matrix(theta):
    '''Construct rotation matrix'''
    c,s    = cos(theta), sin(theta)
    R      = np.matrix(  [[c, -s], [s, c]] )
    return R


def rotate(R, r):
    '''Rotate a position vector r using rotation matrix R'''
    return np.asarray(  (R * np.mat(r).T)  ).flatten()


def get_positions(y, theta):
    '''Compute global marker positions given y and theta'''
    rA     = np.array([0, y])       #global A position
    R      = rotation_matrix(theta)
    rM1    = rA + rotate(R, sM1)    #global M1 position
    rM2    = rA + rotate(R, sM2)    #global M2 position
    return rM1,rM2


def measurement_error(x, q_obs):
    y,theta = x
    rpM1    = q_obs[:2]
    rpM2    = q_obs[2:]
    rM1,rM2 = get_positions(y, theta)
    e1,e2   = rpM1 - rM1, rpM2 - rM2
    e1,e2   = np.linalg.norm(e1), np.linalg.norm(e2)
    f       = e1**2 + e2**2
    return f


def solution_ls(q_obs):
    x0      = [y_true, theta_true]  #initial (y, theta) guess
    results = optimize.minimize(measurement_error, x0, args=(q_obs,))
    y,theta = results.x
    return y, degrees(theta)


def solution_bayesian(q_obs, prior=1):
    '''
    prior=1 :  vague prior
    prior=2 :  informative prior (using LS solution;  same as in Appendix D)
    prior=3 :  highly informative prior
    '''
    
    if prior==1:    # vague prior
        tau     = pymc.Uniform("tau", 0, 10)
        y       = pymc.Uniform("y", -10, 10)
        theta   = pymc.Uniform("theta", -radians(90), radians(90))
    
    elif prior==2:  # LS-informed prior
        qls     = solution_ls( q_obs )
        yls     = qls[0]              
        thetals = radians( qls[1] )   
        tau     = pymc.Uniform("tau", 0, 2 * 1/(noise_amp**2))
        y       = pymc.Normal("y", yls, 5, value=yls)    
        theta   = pymc.Normal("theta", thetals, radians(10), value=thetals)

    elif prior==3:  # highly informative prior
        tau     = pymc.Uniform("tau", 0, 2 * 1/(noise_amp**2))
        y       = pymc.Normal("y", y_true, 5, value=y_true)
        theta   = pymc.Normal("theta", theta_true, radians(10), value=theta_true)


    @pymc.deterministic
    def observations_model(y=y, theta=theta):
        rM1,rM2 = get_positions(y, theta)
        q       = np.asarray([rM1, rM2]).flatten()
        return q
    q_model   = pymc.Normal("q", observations_model, tau, value=q_obs, observed=True)

    mcmc      = pymc.MCMC([q_model, y, theta, tau])
    mcmc.sample(20000, 5000, progress_bar=False)
    Y         = mcmc.trace('y')[:]
    THETA     = np.degrees( mcmc.trace('theta')[:] )
    
    return Y.mean(), THETA.mean()



<br>
<br>

Let's next calculate Bayesian solutions for all three sets of priors in `solution_bayesian` using the simulated the dataset from Appendix D.

<br>
<br>

In [2]:
### local positions:
sM1        = np.array([35.0, 0.0])  # cm
sM2        = np.array([45.0, 0.0])  # cm

### true values:
y_true      = -0.1
theta_true  = radians(5)

### true marker positions:
rM1,rM2    = get_positions(y_true, theta_true) 
q_true     = np.array([rM1, rM2]).flatten()

### measurements:
rpM1       = np.array([33.51, 12.11])
rpM2       = np.array([42.63, 16.18])

### simulation parameters:
np.random.seed(0)
nIterations = 10
noise_amp   = 0.05  # cm
Q_obs       = q_true + noise_amp * np.random.randn(nIterations, 4)


### run simulation:
RESULTS_LS  = np.array( [solution_ls(q_obs)  for q_obs in Q_obs] )
RESULTS_B1  = np.zeros( (nIterations,2) )
RESULTS_B2  = np.zeros( (nIterations,2) )
RESULTS_B3  = np.zeros( (nIterations,2) )
for i,q_obs in enumerate(Q_obs):
    print('Iteration %d of %d...' %(i+1, nIterations))
    y1,theta1 = solution_bayesian(q_obs, prior=1)
    y2,theta2 = solution_bayesian(q_obs, prior=2)
    y3,theta3 = solution_bayesian(q_obs, prior=3)
    RESULTS_B1[i] = [y1, theta1]
    RESULTS_B2[i] = [y2, theta2]
    RESULTS_B3[i] = [y3, theta3]
    print('   y1 = %.3f,  theta1 = %.3f' %(y1,theta1))
    print('   y2 = %.3f,  theta2 = %.3f' %(y2,theta2))
    print('   y3 = %.3f,  theta3 = %.3f' %(y3,theta3))
    print()

Iteration 1 of 10...
   y1 = -0.040,  theta1 = 5.014
   y2 = -0.053,  theta2 = 5.026
   y3 = -0.075,  theta3 = 5.056

Iteration 2 of 10...
   y1 = 0.397,  theta1 = 4.255
   y2 = -0.061,  theta2 = 4.904
   y3 = -0.007,  theta3 = 4.823

Iteration 3 of 10...
   y1 = 0.214,  theta1 = 4.628
   y2 = -0.165,  theta2 = 5.161
   y3 = -0.193,  theta3 = 5.200

Iteration 4 of 10...
   y1 = 1.005,  theta1 = 3.437
   y2 = -0.038,  theta2 = 4.928
   y3 = -0.006,  theta3 = 4.882

Iteration 5 of 10...
   y1 = 0.131,  theta1 = 4.629
   y2 = 0.182,  theta2 = 4.558
   y3 = -0.035,  theta3 = 4.867

Iteration 6 of 10...
   y1 = 0.745,  theta1 = 3.797
   y2 = -0.054,  theta2 = 4.928
   y3 = -0.078,  theta3 = 4.963

Iteration 7 of 10...
   y1 = 0.161,  theta1 = 4.562
   y2 = -0.100,  theta2 = 4.942
   y3 = -0.084,  theta3 = 4.920

Iteration 8 of 10...
   y1 = 0.065,  theta1 = 4.819
   y2 = 0.269,  theta2 = 4.536
   y3 = 0.156,  theta3 = 4.695

Iteration 9 of 10...
   y1 = 0.391,  theta1 = 4.235
   y2 = -0.529

<br>
<br>

Last let's summarize the error from the various methods.

<br>
<br>

In [3]:
x_true   = [y_true, degrees(theta_true)]
error_LS = RESULTS_LS - x_true
error_B1 = RESULTS_B1 - x_true
error_B2 = RESULTS_B2 - x_true
error_B3 = RESULTS_B3 - x_true

print('Average absolute error (least-squares):')
print('   y: %.3f,  theta: %.3f' %tuple( np.abs(error_LS).mean(axis=0) ) )
print('Average absolute error (Bayesian prior = 1):')
print('   y: %.3f,  theta: %.3f' %tuple( np.abs(error_B1).mean(axis=0) ) )
print('Average absolute error (Bayesian prior = 2):')
print('   y: %.3f,  theta: %.3f' %tuple( np.abs(error_B2).mean(axis=0) ) )
print('Average absolute error (Bayesian prior = 3):')
print('   y: %.3f,  theta: %.3f' %tuple( np.abs(error_B3).mean(axis=0) ) )

Average absolute error (least-squares):
   y: 0.170,  theta: 0.232
Average absolute error (Bayesian prior = 1):
   y: 0.425,  theta: 0.603
Average absolute error (Bayesian prior = 2):
   y: 0.169,  theta: 0.242
Average absolute error (Bayesian prior = 3):
   y: 0.131,  theta: 0.195


# Summary

These results show that:

- Using a vague prior (prior=1) for Baysian IK produces poor results relative to LS-IK results.
- Using the LS-IK solution to inform Bayesian priors (prior=2) provides marginal-to-no accuracy improvement. 
- Using highly informative priors (prior=3) yields highly accurate Bayesian IK results; however, since the true solutions are unknown in practice (if they were, IK wouldn't be necessary), this approach is irrelevant to real (non-simulated) IK problems.

In summary, Bayesian IK accuracy is directly related to the information content of the priors. Using an LS-informed prior is not expected to yield accuracy improvements.